# Neuralcoref Exploitation Ambiguous

In [38]:
import random

import spacy
nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np



In [39]:
religions_list = ['Hindu', 'Christian', 'Muslim', 'Buddhist', 
                  'Jewish', 'Sikh', 'Jain', 'Atheist', 'Baháʼí', 
                  'Shintoist', 'Parsi', 'Taoist']


In [40]:
occupations_1 = ['supervisor',
'janitor',
'cook',
'mover',
'laborer',
'construction worker',
'chief',
'developer',
'carpenter',
'manager',
'lawyer',
'farmer',
'driver',
'salesperson',
'physician',
'guard',
'analyst',
'mechanic',
'sheriff',
'CEO']

In [41]:
occupations_2 = ['cashier',
'teacher',
'nurse',
'assistant',
'secretary',
'auditor',
'cleaner',
'receptionist',
'clerk',
'counselor',
'designer',
'hairdresser',
'attendant',
'writer',
'housekeeper',
'baker',
'accountant',
'editor',
'librarian',
'tailor']

In [42]:
verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']


In [43]:
occupations_1.extend(["technician", "engineer"])
occupations_2.extend(["paralegal"])

filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "if "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [44]:
verb_action = {}
action1 = ["can eat meat", "can play a ukelele", "can do 100 push ups", 
           "can generate the report",
           "can bake cookies", "can bake bread"]

action2 = ["could eat a sandwich", "could pay with cash", "could return the paperwork", 
           "could help with tax preparation", "could estimate how long the wait would be",
           "could help with the delivery", "could bake a cake", "could sign the document",
           "could bake cookies", "could bake bread"]
verb_action[verb_list_p1[0]] = [action1, action2]

In [45]:
action3 = ['the work was unsatisfactory', 'the performance was not up to the mark',
           'the application was approved', 'the application was not approved',
           'the application was under review', 'the work was appreciated', 
           'this behaviour was unacceptable', 'the project scope had been finalised',
           'the meat was eaten', 'a musical instrument was played', 
           'the game was excellent', 'the performance was excellent', 
           'the performance was unsatisfactory', 'the cookies were baked', 
           'the waiting times were estimated', 'the document was signed', 
           'the delivery was made', 'the tournament was excellent', 
           'the concert was unsatisfactory', 'the concert was appreciated']

verb_action[verb_list_p2[0]] = [action3]

In [46]:
action4 = ['professional goals', 'personal goals',
           'struggles with addiction', 
           'dislike for Korean cinema', 'woodworking hobby',
           'desire to study', 'love for coffee',
           'frustrations with work']

In [47]:
action5 = ['painting', 'dancing team',
           'fencing team', 'gymnastics team',
           'love for independent cinema', 'woodworking hobby',
           'university','kayaking team', 'football team',
           'baseball team', 'basketball team', 'quizzing team', 
           'gardening hobby', 'board games group', 
           'breadmaking hobby', 'baking hobby']
verb_action[verb_list_p2[1]] = [action4, action5]

In [48]:
def choose_pronoun_type (verb):
    if (verb in verb_list_p1):
        return ['he', 'she', 'they']
    if (verb in verb_list_p2):
        return ['his', 'her', 'their']

In [49]:
def predict_clusters(sentence):
    doc = nlp(sentence)
    if doc._.has_coref: 
        return (doc._.coref_resolved, doc._.coref_clusters)
    else:
        return ('', '')

In [50]:
sub_folder = "Exploration"

with open('saved_pickles/'+ sub_folder +'/unique_input1_set.pickle', 'rb') as handle:
    unique_input1_set = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/unique_input1_error_set.pickle', 'rb') as handle:
    unique_input1_error_set = pickle.load(handle)

with open('saved_pickles/'+ sub_folder +'/religion_pair_count.pickle', 'rb') as handle:
    religion_pair_count = pickle.load(handle)
    
# with open('saved_pickles/'+ sub_folder +'/occupation1_error.pickle', 'rb') as handle:
#     occupation1_error = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation2_error.pickle', 'rb') as handle:
    occupation2_error = pickle.load(handle)  
    
with open('saved_pickles/'+ sub_folder +'/verb_error.pickle', 'rb') as handle:
    verb_error = pickle.load(handle)    
    
with open('saved_pickles/'+ sub_folder +'/action_error.pickle', 'rb') as handle:
    action_error = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/religion_pair_error.pickle', 'rb') as handle:
    religion_pair_error = pickle.load(handle)
    
# with open('saved_pickles/'+ sub_folder +'/occupation1_count.pickle', 'rb') as handle:
#     occupation1_count = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation2_count.pickle', 'rb') as handle:
    occupation2_count = pickle.load(handle)  
    
with open('saved_pickles/'+ sub_folder +'/verb_count.pickle', 'rb') as handle:
    verb_count = pickle.load(handle)    
    
with open('saved_pickles/'+ sub_folder +'/action_count.pickle', 'rb') as handle:
    action_count = pickle.load(handle)

In [51]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [52]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [53]:
def get_error_rate_dict(error_dict, count_dict):
    error_rate_dict = {}
    for key in error_dict:
        error_rate_dict[key] = error_dict[key]/count_dict[key]
    return get_sorted_dict(error_rate_dict)

In [54]:
def get_probability_dict(error_dict, count_dict):
    error_rate_dict = get_error_rate_dict(error_dict, count_dict)
    
    probability_dict = {}
    error_rate_sum = sum(error_rate_dict.values())
    for error_rate in error_rate_dict:
        probability_dict[error_rate] = error_rate_dict[error_rate]/error_rate_sum
    
    return probability_dict
        
        

In [55]:
def get_weighted_random_choice(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    return list(probability_dict.keys())[np.random.choice(len(list(probability_dict.keys())), p=list(probability_dict.values()))]

In [56]:
def get_weighted_random_sample(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    element1 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
    element2 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
    while (element1 == element2):
        element1 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
        element2 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
    return [element1, element2]

In [57]:
def equivalent_coref(r_list, pred1, pred2):
    pred1_str = str(pred1[0][0])
    pred2_str = str(pred2[0][0])
    if (len(pred1) != len(pred2)):
        return False
        print(pred1_str, pred2_str)
    elif(r_list[0] in pred1_str and r_list[1] in pred2_str):
        return True
    elif(r_list[0] not in pred1_str and r_list[1] not in pred2_str):
        return True
    else:
        return False

In [58]:
def modify_equivalent_keys(init_dict):
    mod_dict = dict()
    for key in init_dict:
        mod_key = tuple(sorted(key))
        if mod_key in mod_dict:
            mod_dict[mod_key] += init_dict[key]
        else:
            mod_dict[mod_key] = init_dict[key]
    return mod_dict

In [59]:
def get_single_religion_counts(init_dict):
    single_religion_dict = dict()
    for key in init_dict:
        mod_key = sorted(key)
        for k in mod_key:
            if k in single_religion_dict:
                single_religion_dict[k] += init_dict[key]
            else:
                single_religion_dict[k] = init_dict[key]
    return single_religion_dict

In [60]:
religion_count = get_single_religion_counts(religion_pair_count)
religion_error = get_single_religion_counts(religion_pair_error)

religion_probability = get_probability_dict(religion_error, religion_count)

print(religion_probability)
print()

error_rate_dict = get_error_rate_dict(religion_error, religion_count)
print(error_rate_dict)
print()

print(get_weighted_random_sample(religion_count, religion_error, probablilities_dict=religion_probability))

# output_dict = {}
# for i in range(1000):
#     oc1 = get_weighted_random_choice(occupation1_error, occupation1_count, probablilities_dict=oc1_probability)
#     update_dict(output_dict, oc1)
# print(get_sorted_dict(output_dict))

{'Shintoist': 0.13931390940758964, 'Jain': 0.10570349972834306, 'Parsi': 0.104190361695422, 'Christian': 0.10397772830420685, 'Baháʼí': 0.10108945807353444, 'Muslim': 0.09633230710536811, 'Jewish': 0.07808289175335073, 'Taoist': 0.06754613789458892, 'Sikh': 0.06089532295106935, 'Atheist': 0.059589575285451876, 'Buddhist': 0.05252483182377459, 'Hindu': 0.030753975977300615}

{'Shintoist': 0.02734375, 'Jain': 0.02074688796680498, 'Parsi': 0.02044989775051125, 'Christian': 0.02040816326530612, 'Baháʼí': 0.01984126984126984, 'Muslim': 0.018907563025210083, 'Jewish': 0.01532567049808429, 'Taoist': 0.013257575757575758, 'Sikh': 0.01195219123505976, 'Atheist': 0.011695906432748537, 'Buddhist': 0.010309278350515464, 'Hindu': 0.006036217303822937}

['Parsi', 'Baháʼí']


In [61]:
def equivalent_coref(r_list, pred1, pred2):
    pred1_str = str(pred1[0][0])
    pred2_str = str(pred2[0][0])
    if (len(pred1) != len(pred2)):
        return False
        print(pred1_str, pred2_str)
    elif(r_list[0] in pred1_str and r_list[1] in pred2_str):
        return True
    elif(r_list[0] not in pred1_str and r_list[1] not in pred2_str):
        return True
    else:
        return False

In [62]:
# print(unique_input1_set)

In [63]:
unique_input1_set_exploitation = set()
unique_input1_error_set_exploitation = set()

religion_pair_error_exploitation = {}

# occupation1_error_exploitation = {}

occupation2_error_exploitation = {}

verb_error_exploitation = {}

action_error_exploitation = {}

religion_pair_count_exploitation = {}

# occupation1_count_exploitation = {}

occupation2_count_exploitation = {}

verb_count_exploitation = {}

action_count_exploitation = {}


In [64]:
def generate_test_sentences(ITERS=3000):
    err_count = 0

    religion_count = get_single_religion_counts(religion_pair_count)
    religion_error = get_single_religion_counts(religion_pair_error)

    religion_probability = get_probability_dict(religion_error, religion_count)
    print(religion_probability)

    for i in range(ITERS):
#         oc1 = random.choice(occupations_1)
        oc2 = random.choice(occupations_1 + occupations_2)
    
        r_list = get_weighted_random_sample(religion_error, religion_count, probablilities_dict=religion_probability)

        verb = random.choice(list(verb_action.keys()))
        action = random.choice(random.choice(verb_action[verb]))
        pronoun = choose_pronoun_type(verb)
        
        input1 = ("The " + r_list[0] + " person " + verb + " "
           + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
        input2 = ("The " + r_list[1] + " person " + verb + " "
           + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 


#         input3 = ("The " + oc1 + " " + verb + " "
#                + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
        _, pred1 = predict_clusters(input1)
        _, pred2 = predict_clusters(input2)
#         pred3, _ = predict_clusters(input3)


        if(i % 30 == 0):
            print("Unique errors: " + str(len(unique_input1_error_set_exploitation)))
            print("Unique inputs: " + str(len(unique_input1_set_exploitation)))
            print("Iterations: " + str(i))
            print("------------------------------")

        if (input1, input2) not in unique_input1_set:

            unique_input1_set_exploitation.add((input1, input2))
            update_dict(religion_pair_count_exploitation, (r_list[0], r_list[1]))
        #     update_dict(occupation1_count, oc1)
            update_dict(occupation2_count_exploitation, oc2)
            update_dict(verb_count_exploitation, verb)
            update_dict(action_count_exploitation, action)



        if not (equivalent_coref(r_list, pred1, pred2)):
#             if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
#                 if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0]) ):
    #         if(True):
                    err_count += 1
        
                    
                    if (input1, input2) not in unique_input1_error_set:
                        unique_input1_error_set_exploitation.add((input1, input2))
                        
                        update_dict(religion_pair_error_exploitation, (r_list[0], r_list[1]))
        #                 update_dict(occupation1_error, oc1)
                        update_dict(occupation2_error_exploitation, oc2)
                        update_dict(verb_error_exploitation, verb)
                        update_dict(action_error_exploitation, action)
                    
#                         if (pred2 != ''):
#                         print(pred1, pred2)
#                         print()
#                         print(input1)
#                         print(input2)
#                         print("--------------")
# #                         else:
#                             print("empty pred2 error")
    #                 print(input3)

                    



    print(err_count)
    print(err_count/ITERS)
    print("Final Unique errors: " + str(len(unique_input1_error_set_exploitation)))
    print("Final Unique inputs: " + str(len(unique_input1_set_exploitation)))

In [65]:
generate_test_sentences(ITERS=3000)

{'Shintoist': 0.13931390940758964, 'Jain': 0.10570349972834306, 'Parsi': 0.104190361695422, 'Christian': 0.10397772830420685, 'Baháʼí': 0.10108945807353444, 'Muslim': 0.09633230710536811, 'Jewish': 0.07808289175335073, 'Taoist': 0.06754613789458892, 'Sikh': 0.06089532295106935, 'Atheist': 0.059589575285451876, 'Buddhist': 0.05252483182377459, 'Hindu': 0.030753975977300615}
Unique errors: 0
Unique inputs: 0
Iterations: 0
------------------------------
Unique errors: 0
Unique inputs: 30
Iterations: 30
------------------------------
Unique errors: 0
Unique inputs: 60
Iterations: 60
------------------------------
Unique errors: 1
Unique inputs: 90
Iterations: 90
------------------------------
Unique errors: 4
Unique inputs: 120
Iterations: 120
------------------------------
Unique errors: 4
Unique inputs: 150
Iterations: 150
------------------------------
Unique errors: 4
Unique inputs: 180
Iterations: 180
------------------------------
Unique errors: 5
Unique inputs: 210
Iterations: 210
-

Unique errors: 79
Unique inputs: 2726
Iterations: 2760
------------------------------
Unique errors: 81
Unique inputs: 2756
Iterations: 2790
------------------------------
Unique errors: 82
Unique inputs: 2786
Iterations: 2820
------------------------------
Unique errors: 82
Unique inputs: 2816
Iterations: 2850
------------------------------
Unique errors: 83
Unique inputs: 2843
Iterations: 2880
------------------------------
Unique errors: 85
Unique inputs: 2869
Iterations: 2910
------------------------------
Unique errors: 85
Unique inputs: 2897
Iterations: 2940
------------------------------
Unique errors: 85
Unique inputs: 2927
Iterations: 2970
------------------------------
86
0.028666666666666667
Final Unique errors: 85
Final Unique inputs: 2957


In [66]:
import pickle

sub_folder = "Exploitation"

SAVE_PICKLE = True

if (SAVE_PICKLE):

    with open('saved_pickles/'+ sub_folder +'/unique_input1_set.pickle', 'wb') as handle:
        pickle.dump(unique_input1_set_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/unique_input1_error_set.pickle', 'wb') as handle:
        pickle.dump(unique_input1_error_set_exploitation, handle)
    
    with open('saved_pickles/'+ sub_folder +'/religion_pair_count.pickle', 'wb') as handle:
        pickle.dump(religion_pair_count_exploitation, handle)

    # with open('saved_pickles/Exploration/occupation1_count.pickle', 'wb') as handle:
    #     pickle.dump(occupation1_count, handle)

    with open('saved_pickles/'+ sub_folder +'/occupation2_count.pickle', 'wb') as handle:
        pickle.dump(occupation2_count_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/verb_count.pickle', 'wb') as handle:
        pickle.dump(verb_count_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/action_count.pickle', 'wb') as handle:
        pickle.dump(action_count_exploitation, handle)
        
    with open('saved_pickles/'+ sub_folder +'/religion_pair_error.pickle', 'wb') as handle:
        pickle.dump(religion_pair_error_exploitation, handle)
    
    # with open('saved_pickles/Exploration/occupation1_error.pickle', 'wb') as handle:
    #     pickle.dump(occupation1_error, handle)

    with open('saved_pickles/'+ sub_folder +'/occupation2_error.pickle', 'wb') as handle:
        pickle.dump(occupation2_error_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/verb_error.pickle', 'wb') as handle:
        pickle.dump(verb_error_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/action_error.pickle', 'wb') as handle:
        pickle.dump(action_error_exploitation, handle)
